In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, learning_curve, validation_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, precision_recall_curve, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import joblib
import seaborn as sns

In [ ]:
# Загрузка данных из CSV файла
data = pd.read_csv('loan_approval_dataset.csv')

In [ ]:
# Вывод информации о данных и первых пяти строках для проверки
data.info()
data.head()

In [ ]:
# Удаление пробелов и приведение к нижнему регистру для категориальных переменных
data['education'] = data['education'].str.strip().str.lower()
data['self_employed'] = data['self_employed'].str.strip().str.lower()
data['loan_status'] = data['loan_status'].str.strip().str.lower()

In [ ]:
# Преобразование категориальных переменных в числовые значения
data['education'] = data['education'].map({'graduate': 1, 'not graduate': 0})
data['self_employed'] = data['self_employed'].map({'yes': 1, 'no': 0})
data['loan_status'] = data['loan_status'].map({'approved': 1, 'rejected': 0})

# Выделение признаков (X) и целевой переменной (y)
X = data.drop(['loan_id', 'loan_status'], axis=1)
y = data['loan_status']

# Разделение данных на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Создание модели градиентного бустинга
model = GradientBoostingClassifier(random_state=42)

# Обучение модели на тренировочных данных
model.fit(X_train, y_train)

In [ ]:
# Предсказания на тестовых данных
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Вывод отчета о классификации
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Вывод матрицы ошибок
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Построение ROC-кривой
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f"Gradient Boosting (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'r--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()

Что показывает:

- ROC-кривая показывает соотношение между True Positive Rate (TPR) и False Positive Rate (FPR) при различных порогах классификации.
- AUC (Area Under the Curve) является мерой качества модели: чем ближе AUC к 1, тем лучше модель различает - График помогает понять, насколько хорошо модель различает положительные и отрицательные классы.

In [ ]:
# Построение графика матрицы ошибок
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Rejected', 'Approved'], yticklabels=['Rejected', 'Approved'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

Что показывает:

- Матрица ошибок показывает количество истинных положительных (True Positives, TP), истинных отрицательных (True Negatives, TN), ложных положительных (False Positives, FP) и ложных отрицательных (False Negatives, FN) предсказаний.
- График помогает визуализировать, как часто модель правильно и неправильно классифицирует образцы.


In [ ]:
# Построение Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.show()

Что показывает:

- Precision-Recall Curve показывает соотношение между точностью (Precision) и полнотой (Recall) при различных порогах классификации.
- График особенно полезен для несбалансированных данных, так как показывает, насколько хорошо модель справляется с положительными классами.


In [ ]:
# Визуализация важности признаков
feature_importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
plt.barh(features, feature_importances, color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importances')
plt.gca().invert_yaxis()  # Инвертируем ось, чтобы самый важный был сверху
plt.show()

Что показывает:

- График показывает важность каждого признака для модели.
- Помогает понять, какие признаки наиболее влияют на предсказания модели.


In [ ]:
# Построение кривой обучения
train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10))

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training Score', marker='o')
plt.plot(train_sizes, test_mean, label='Validation Score', marker='o')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curve')
plt.legend()
plt.grid()
plt.show()

Что показывает:

- Кривая обучения показывает, как изменяется точность модели на тренировочных и валидационных данных при увеличении размера тренировочного набора.
- Помогает понять, насколько хорошо модель обучается и обобщает данные.


In [ ]:
# Построение кривой валидации для гиперпараметра n_estimators
param_range = np.linspace(50, 500, 5, dtype=int)
train_scores, test_scores = validation_curve(
    GradientBoostingClassifier(random_state=42),
    X, y, param_name='n_estimators', param_range=param_range, cv=5, scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(param_range, train_mean, label='Training Score', marker='o')
plt.plot(param_range, test_mean, label='Validation Score', marker='o')
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy')
plt.title('Validation Curve (n_estimators)')
plt.legend()
plt.grid()
plt.show()

Что показывает:

- Кривая валидации показывает, как изменяется точность модели на тренировочных и валидационных данных при изменении значения гиперпараметра (в данном случае n_estimators).
- Помогает понять, как гиперпараметры влияют на производительность модели.


In [ ]:
# Построение графика накопительных приростов
total = len(y_test)
positive = sum(y_test)
sorted_probas = sorted(zip(y_test, y_pred_proba), key=lambda x: x[1], reverse=True)
cumulative_positive = np.cumsum([x[0] for x in sorted_probas])

plt.figure(figsize=(8, 6))
plt.plot(np.arange(total) / total, cumulative_positive / positive, label='Model', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random', color='red')
plt.xlabel('Percentage of Samples')
plt.ylabel('Percentage of Positives')
plt.title('Cumulative Gains Chart')
plt.legend()
plt.grid()
plt.show()

Что показывает:

- График накопительных приростов показывает, какую долю положительных образцов модель находит при увеличении доли образцов, отсортированных по убыванию предсказанных вероятностей.
- Помогает понять, насколько хорошо модель различает положительные образцы.


In [ ]:
# Построение гистограммы распределения вероятностей предсказания классов
plt.figure(figsize=(8, 6))
plt.hist(y_pred_proba[y_test == 1], bins=20, alpha=0.7, label='Approved', color='green')
plt.hist(y_pred_proba[y_test == 0], bins=20, alpha=0.7, label='Rejected', color='red')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.legend()
plt.show()

Что показывает:

- Гистограмма распределения вероятностей показывает распределение предсказанных вероятностей для положительных и отрицательных классов.
- Помогает понять, как модель распределяет вероятности предсказания для разных классов.


In [ ]:
# Сохранение обученной модели в файл
joblib.dump(model, 'gradient_boosting_model.pkl')

In [ ]:
# Ввод собственных данных для предсказания
custom_data = {
    'no_of_dependents': [3],
    'education': [1],  # 1 = Graduate, 0 = Not Graduate
    'self_employed': [0],  # 1 = Yes, 0 = No
    'income_annum': [7000000],
    'loan_amount': [20000000],
    'loan_term': [15],
    'cibil_score': [750],
    'residential_assets_value': [5000000],
    'commercial_assets_value': [3000000],
    'luxury_assets_value': [15000000],
    'bank_asset_value': [8000000]
}

# Преобразование данных в DataFrame
custom_data_df = pd.DataFrame(custom_data)

# Загрузка ранее сохранённой модели
loaded_model = joblib.load('gradient_boosting_model.pkl')

# Предсказание для пользовательских данных
custom_prediction = loaded_model.predict(custom_data_df)
custom_prediction_proba = loaded_model.predict_proba(custom_data_df)

# Вывод результатов предсказания
status = 'Одобрено' if custom_prediction[0] == 1 else 'Отклонено'
print(f"Результат предсказания для пользовательских данных: {status}")
print(f"Вероятность: {custom_prediction_proba[0][1]:.2f} для одобрения")

In [ ]:
# Ввод собственных данных
custom_data = {
    'no_of_dependents': [5],  # Много иждивенцев
    'education': [0],  # Нет высшего образования
    'self_employed': [1],  # Самозанятый
    'income_annum': [500000],  # Низкий доход
    'loan_amount': [10000000],  # Высокая сумма кредита
    'loan_term': [20],  # Длительный срок кредита
    'cibil_score': [400],  # Низкий кредитный рейтинг
    'residential_assets_value': [500000],  # Низкая стоимость жилых активов
    'commercial_assets_value': [200000],  # Низкая стоимость коммерческих активов
    'luxury_assets_value': [100000],  # Отсутствие предметов роскоши
    'bank_asset_value': [100000]  # Маленький вклад в банке
}

# Преобразование данных в DataFrame
custom_data_df = pd.DataFrame(custom_data)

# Загрузка ранее сохранённой модели
loaded_model = joblib.load('gradient_boosting_model.pkl')

# Предсказание
custom_prediction = loaded_model.predict(custom_data_df)
custom_prediction_proba = loaded_model.predict_proba(custom_data_df)

# Результаты
status = 'Одобрено' if custom_prediction[0] == 1 else 'Отклонено'
print(f"Результат предсказания для пользовательских данных: {status}")
print(f"Вероятность: {custom_prediction_proba[0][1]:.2f} для одобрения")